# Collect RAWS Data with SynopticPy

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

from synoptic.services import stations_timeseries, stations_metadata

## Station Time Series

In [ ]:
params = dict(
    stid=["KAFF"],
    vars=["air_temp", "relative_humidity", "precip_accum", "fuel_moisture", "wind_speed", "solar_radiation"],
    start=datetime(2023, 2, 1),
    end=datetime(2024, 1, 31),
)

a = stations_timeseries(**params)

a

In [ ]:
type(a)

In [ ]:
print(f'Station Elevation: {a.attrs["ELEVATION"]}')
print(f'Lat: {a.attrs["latitude"]}')
print(f'Lon: {a.attrs["longitude"]}')
print(f'Temp Units: {a.attrs["UNITS"]["air_temp"]}')

In [ ]:
a.attrs

In [ ]:
a.attrs["UNITS"]["air_temp"] == "Celsius"

In [ ]:
a.attrs["UNITS"]['elevation']

## Get Data For Bounding Box

For any moderate large bounding box, the package returns errors that too many station hours were requested. We will explore getting a paid token to get around this, but for now we will loop over available stations in the bounding box to collect data.

In [ ]:
box = [-105, 37, -103, 39]
# Find all stations with fuel moisture
sts = stations_metadata(
    bbox=box,
    vars=["fuel_moisture"]
)

In [ ]:
sts

In [ ]:
params = dict(
    stid=["PLACEHOLDER"], # change this in the loop
    vars=["air_temp", "relative_humidity", "precip_accum", "fuel_moisture", "wind_speed", "solar_radiation"],
    start=datetime(2023, 6, 1),
    end=datetime(2023, 6, 30),
)

In [ ]:
# Set up final data dictionary
raws_dict = {}

In [ ]:
def format_raws_df(df):
    # Given input dataframe (the output of retrieve_raws_api), return formatted dictionary
    # Inputs:
    # df: (dataframe)
    # Returns: fmda dictionary

    ## Format Return Dictionaries
    loc = {
        "STID": df.attrs["STID"],
        'lat' : df.attrs['latitude'],
        'lon' : df.attrs['longitude'],
        'elev': df.attrs["ELEVATION"]
    }
    
    ## Extract times from dataframe index
    times = df.index.strftime('%Y-%m-%dT%H:%M:%SZ').to_numpy() # convert index to utc time
    ## Convert dataframe to dictionary
    raws = df.to_dict(orient = "list")
    
    # Convert lists to NumPy arrays
    raws = {key: np.array(value) for key, value in raws.items()}

    raws["time_raws"]=times
    raws["hours"]=len(times)
    
    ## Convert C to K 
    if df.attrs["UNITS"]["air_temp"] == "Celsius":
        print("Converting RAWS temp from C to K")
        raws["air_temp"] = raws["air_temp"]+273.15

    return loc, raws

In [ ]:
for st in sts:
    print("~"*50)
    print(f"Collecting data for {st}")
    params["stid"] = [st]
    try:
        dat = stations_timeseries(verbose="HIDE", **params)

        if "fuel_moisture" in dat.columns:
            print("Collected FMC data")
            loc, raws = format_raws_df(dat)
            raws_dict[st] = {"loc":loc, "RAWS": raws}
        else:
            print("No FMC found for this station and time")
    except AssertionError as e:
        # Error handling behavior
        print("AssertionError caught:", e)
        # Other behavior you want to execute if AssertionError is caught


In [ ]:
# Write raws_dict pickle file use for testing in other areas.
import pickle

with open('test_raws_dict.pickle', 'wb') as handle:
    pickle.dump(raws_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)